In [78]:
import sys
import ast
import pandas as pd
import numpy as np

In [79]:
listOfEvents = []
input_file = "out2.txt"
with open(input_file,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df = pd.DataFrame.from_dict(listOfEvents)
# df["ReadAddr"]=""
# df["WriteAddr"]=""
df["AllocatedPages"]=""
# df["AllocatedIDs"]=""



In [80]:

df.dtypes

starttime           int64
address            object
size               object
type               object
cudatime           object
cudatimeoverlap    object
eventid             int64
AllocatedPages     object
dtype: object

In [81]:
# df.loc[df["kernelid"]==0,["ReadAddr"]]="anything"
# df[df["kernelid"]==0]["ReadAddr"]

In [82]:
nvbit_file = "../mem_trace_multilayer.txt"
with open(nvbit_file,"r") as f:
    for line in f:
        if "Read" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            readAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(readAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["ReadAddr"]]=str(readAddr)
        if "Write" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            writeAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(writeAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["WriteAddr"]]=str(writeAddr)
            


In [83]:
listOfEvents = []
input_file_nvbit = "out2_nvbit.txt"
with open(input_file_nvbit,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df_realaddr = pd.DataFrame.from_dict(listOfEvents)

In [84]:
realaddr = df_realaddr["address"]
for i, ra in enumerate(realaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["address"]]=ra

In [85]:
def get_pagesAllocated(addr, size):
    if size < 0:
        return ""
    else:
        pages_to_return = []
        maxAddr = addr + size
        pageAddr = addr // 4096 *4096
        while pageAddr <= maxAddr:
            pages_to_return.append(hex(pageAddr))
            pageAddr += 4096
    return pages_to_return

In [86]:
df_realaddr['address'] = df_realaddr['address'].apply(lambda x:int(x,0))
df_realaddr['size'] = df_realaddr['size'].astype('int')
df_realaddr[['address','size']]


,address,size
0,1476130688,8
1,1476227904,8
2,1460523904,400000
3,1476230848,8
4,1476230848,-8
...,...,...
77,140133984632832,2048
78,140133961826304,1644032
79,140133961826304,-1644032
80,140136456139264,512


In [87]:
df_realaddr['col_3'] = df_realaddr.apply(lambda x: str(get_pagesAllocated(x['address'], x['size'])), axis=1)

In [88]:
df_realaddr['col_3'].isnull().sum()

0

In [89]:
pageaddr = df_realaddr["col_3"]
for i, pa in enumerate(pageaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["AllocatedPages"]]=pa

In [90]:
df.to_csv(input_file.split('.')[0]+"2.csv")

In [92]:
def convert_addr_to_id(mapping,curid,addr_list):
    print(type(addr_list))
    # for p in addr_list:
    #     id = mapping.get(p)
    #     if id == None:
    #         mapping[p]=curid
    #         curid += 1

In [96]:
mapping = {}
curid = 0
for index, row in df.iterrows():
    if row['AllocatedPages'] != "":
        print(index, ',',row['AllocatedPages'])
        page_list = ast.literal_eval(row['AllocatedPages'])
        id_list = []
        for p in page_list:
            if p[:4] == "0x7f":
                id = mapping.get(p)
                if id == None:
                    mapping[p]=curid
                    id_list.append(curid)
                    curid += 1
                else:
                    id_list.append(id)

        df.loc[index,'AllocatedIDs']=str(id_list)

0 , ['0x57fbf000']
1 , ['0x57fd7000']
2 , ['0x570dd000', '0x570de000', '0x570df000', '0x570e0000', '0x570e1000', '0x570e2000', '0x570e3000', '0x570e4000', '0x570e5000', '0x570e6000', '0x570e7000', '0x570e8000', '0x570e9000', '0x570ea000', '0x570eb000', '0x570ec000', '0x570ed000', '0x570ee000', '0x570ef000', '0x570f0000', '0x570f1000', '0x570f2000', '0x570f3000', '0x570f4000', '0x570f5000', '0x570f6000', '0x570f7000', '0x570f8000', '0x570f9000', '0x570fa000', '0x570fb000', '0x570fc000', '0x570fd000', '0x570fe000', '0x570ff000', '0x57100000', '0x57101000', '0x57102000', '0x57103000', '0x57104000', '0x57105000', '0x57106000', '0x57107000', '0x57108000', '0x57109000', '0x5710a000', '0x5710b000', '0x5710c000', '0x5710d000', '0x5710e000', '0x5710f000', '0x57110000', '0x57111000', '0x57112000', '0x57113000', '0x57114000', '0x57115000', '0x57116000', '0x57117000', '0x57118000', '0x57119000', '0x5711a000', '0x5711b000', '0x5711c000', '0x5711d000', '0x5711e000', '0x5711f000', '0x57120000', '0x57

In [98]:
df.loc[0:10,'AllocatedIDs']

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4                                                   NaN
5     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
6                                                  [96]
7     [96, 98, 99, 100, 101, 102, 103, 104, 105, 106...
8                                                   NaN
9                                                   NaN
10    [243, 245, 246, 247, 248, 249, 250, 251, 252, ...
Name: AllocatedIDs, dtype: object